In [6]:
import os 
import re
import numpy as np
import json
import random
import copy

data_path_1 = '../data/train_fine.txt'
data_path_2 = '../data/train_coarse.txt'

data_pos_list = []
data_neg_list = []

with open(data_path_1, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        data = json.loads(line)
        title = data['title']
        title = re.sub(r'\d{1,4}年','', title)
        data['title'] = title
        if data['match']['图文'] == 1:
            data_pos_list.append(data)
        else:
            data_neg_list.append(data)

with open(data_path_2, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        data = json.loads(line)
        title = data['title']
        title = re.sub(r'\d{1,4}年','', title)
        data['title'] = title
        if data['match']['图文'] == 1:
            data_pos_list.append(data)
        else:
            data_neg_list.append(data)

new_neg_list = []
for i in range(len(data_pos_list)):
    dic = copy.deepcopy(data_pos_list[i])
    index = random.randint(0, len(data_pos_list)-1)
    while index == i:
        index = random.randint(0, len(data_pos_list)-1)
    dic['title'] = data_pos_list[index]['title']
    dic['match']['图文'] = 0
    new_neg_list.append(dic)

print(len(data_pos_list), len(data_neg_list), len(new_neg_list))

139588 10412 139588


In [11]:
all_data = data_pos_list + data_neg_list + new_neg_list
np.random.shuffle(all_data)
l = int(len(all_data) * 0.9)
x_train_list = all_data[:l]
x_val_list = all_data[l:]
# np.random.shuffle(x_val_list)

x_train_list = [json.dumps(dic, ensure_ascii=False)+'\n' for dic in x_train_list]
x_val_list = [json.dumps(dic, ensure_ascii=False)+'\n' for dic in x_val_list]


# with open('./data/pretrain_match.txt', 'w', encoding='utf-8') as f:
#     f.writelines(pre_ret)

with open('../data/preprocessed_data/new_train_match.txt', 'w', encoding='utf-8') as f:
    f.writelines(x_train_list)

with open('../data/preprocessed_data/new_val_match.txt', 'w', encoding='utf-8') as f:
    f.writelines(x_val_list)

# print(len(pre_ret))
print(len(x_train_list))
print(len(x_val_list))
print(len(x_train_list) + len(x_val_list))

KeyboardInterrupt: 

In [2]:
import json
new_dic = {}

with open('../data/attr_to_attrvals.json', 'r', encoding='utf-8') as f:
    attr_key = json.load(f)
    for key, value in attr_key.items():
        tmp = []
        for v in value:
            if '=' in v:
                tmp.append(v.split('='))
            else:
                tmp.append([v])
        new_dic[key] = tmp
print(new_dic)
rets = [json.dumps(new_dic, ensure_ascii=False)+'\n']
with open('../data/preprocessed_data/attr_match.json', 'w', encoding='utf-8') as f:
    f.writelines(rets)

{'领型': [['高领', '半高领', '立领'], ['连帽', '可脱卸帽'], ['翻领', '衬衫领', 'POLO领', '方领', '娃娃领', '荷叶领'], ['双层领'], ['西装领'], ['U型领'], ['一字领'], ['围巾领'], ['堆堆领'], ['V领'], ['棒球领'], ['圆领'], ['斜领'], ['亨利领']], '袖长': [['短袖', '五分袖'], ['九分袖', '长袖'], ['七分袖'], ['无袖']], '衣长': [['超短款', '短款', '常规款'], ['长款', '超长款'], ['中长款']], '版型': [['修身型', '标准型'], ['宽松型']], '裙长': [['短裙', '超短裙'], ['中裙', '中长裙'], ['长裙']], '穿着方式': [['套头'], ['开衫']], '类别': [['手提包'], ['单肩包'], ['斜挎包'], ['双肩包']], '裤型': [['O型裤', '锥形裤', '哈伦裤', '灯笼裤'], ['铅笔裤', '直筒裤', '小脚裤'], ['工装裤'], ['紧身裤'], ['背带裤'], ['喇叭裤', '微喇裤'], ['阔腿裤']], '裤长': [['短裤'], ['五分裤'], ['七分裤'], ['九分裤', '长裤']], '裤门襟': [['松紧'], ['拉链'], ['系带']], '闭合方式': [['松紧带'], ['拉链'], ['套筒', '套脚', '一脚蹬'], ['系带'], ['魔术贴'], ['搭扣']], '鞋帮高度': [['高帮', '中帮'], ['低帮']]}


In [3]:
with open('../data/preprocessed_data/attr_match.json', 'r', encoding='utf-8') as f:
    attr_key = json.load(f)

for key, value in attr_key.items():
    att = set()
    for i in value:
        for j in i:
            att.add(j)
    print(f'key type: {len(value)} num:{len(att)}')

key type: 14 num:22
key type: 4 num:6
key type: 3 num:6
key type: 2 num:3
key type: 3 num:5
key type: 2 num:2
key type: 4 num:4
key type: 7 num:13
key type: 4 num:5
key type: 3 num:3
key type: 6 num:8
key type: 2 num:3


In [43]:
import json
import numpy as np
from random import choice

with open('../data/preprocessed_data/attr_match.json', 'r', encoding='utf-8') as f:
    attr_key = json.load(f)

def get_neg_item(ori_item, attr_key):
    new_item = copy.deepcopy(ori_item)
    keys = [x for x in new_item['match'].keys() if x != '图文']
    if len(keys) == 0:
        return None
    change_num = 0
    rates = (np.random.random(len(keys))>0.7)
    while np.sum(rates)==0:
        rates = (np.random.random(len(keys))>0.7)
    for key, rate in zip(keys,rates):
        if rate>0.6:
            all_key_values = attr_key[key]
            ori_value = new_item['key_attr'][key]
            key_index = 0
            for i in range(len(all_key_values)):
                if ori_value in all_key_values[i]:
                    key_index = i
                    break
            new_index = np.random.randint(len(all_key_values))
            while new_index == key_index:
                new_index = np.random.randint(len(all_key_values))
            sub_val = all_key_values[new_index]
            new_value = sub_val[np.random.randint(len(sub_val))]
            new_item['title'] = new_item['title'].replace(ori_value, new_value, 1)
            new_item['key_attr'][key] = new_value
            new_item['match'][key] = 0
            change_num+=1
    if change_num>0:
        new_item['match']['图文'] = 0
        return new_item
    else:
        return None

def get_pos_item(ori_item, attr_key):
    new_item = copy.deepcopy(ori_item)
    keys = [x for x in new_item['match'].keys() if x != '图文']
    if len(keys) == 0:
        return None
    change_num = 0
    for key in keys:
        all_key_values = attr_key[key]
        # print(keys, key, new_item['key_attr'])
        ori_value = new_item['key_attr'][key]
        key_index = 0
        for i in range(len(all_key_values)):
            if ori_value in all_key_values[i]:
                key_index = i
                break
        sim_key_values = all_key_values[key_index]
        if len(sim_key_values)>1:
            new_value = ori_value
            while new_value == ori_value:
                new_value = sim_key_values[np.random.randint(len(sim_key_values))]
            new_item['title'] = new_item['title'].replace(ori_value, new_value, 1)
            new_item['key_attr'][key] = new_value
            change_num+=1
    if change_num>0:
        return new_item
    else:
        return None
    

    
# def get_random_key(keys, ratio=0.7):
#     list_ratio = [0.7, 0.9, 1]
#     ratio = choice(list_ratio)
#     l = int(len(keys) * ratio)
#     if l == 0:
#         l = 1
#     np.random.shuffle(keys)
#     return keys[:l]

count = 0
math_list = []
not_math_list = []
with open('../data/train_fine.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    new_dic = {}
    for line in lines:
        data = json.loads(line)
        # math_list.append(json.dumps(data, ensure_ascii=False)+'\n')
        new_item = get_pos_item(data, attr_key)
        new_item is not None and math_list.append(json.dumps(new_item, ensure_ascii=False)+'\n')
        new_item = get_neg_item(data, attr_key)
        new_item is not None and not_math_list.append(json.dumps(new_item, ensure_ascii=False)+'\n')
        # new_item = get_neg_item(data, attr_key)
        # new_item is not None and not_math_list.append(json.dumps(new_item, ensure_ascii=False)+'\n')
        count+=1
        if count >20:
            break

with open('../data/preprocessed_data/my_pos_fine.txt', 'w', encoding='utf-8') as f:
    f.writelines(math_list)
with open('../data/preprocessed_data/my_neg_fine.txt', 'w', encoding='utf-8') as f:
    f.writelines(not_math_list)

TypeError: 'NoneType' object is not subscriptable

In [4]:
import json
import numpy as np
import copy
import re
from random import choice

with open('../data/preprocessed_data/attr_match.json', 'r', encoding='utf-8') as f:
    attr_key = json.load(f)

def get_neg_item(ori_item, attr_key):
    new_item = copy.deepcopy(ori_item)
    keys = [x for x in new_item['match'].keys() if x != '图文']
    if len(keys) == 0:
        return None
    change_num = 0
    rates = (np.random.random(len(keys))>0.7)
    while np.sum(rates)==0:
        rates = (np.random.random(len(keys))>0.7)
    for key, rate in zip(keys,rates):
        if rate:
            all_key_values = attr_key[key]
            ori_value = new_item['key_attr'][key]
            key_index = 0
            for i in range(len(all_key_values)):
                if ori_value in all_key_values[i]:
                    key_index = i
                    break
            new_index = np.random.randint(len(all_key_values))
            while new_index == key_index:
                new_index = np.random.randint(len(all_key_values))
            sub_val = all_key_values[new_index]
            new_value = sub_val[0]
            new_item['title'] = new_item['title'].replace(ori_value, new_value, 1)
            new_item['key_attr'][key] = new_value
            new_item['match'][key] = 0
            change_num+=1
    if change_num>0:
        new_item['match']['图文'] = 0
        return new_item
    else:
        return None

def get_uni_pos_item(ori_item, attr_key):
    new_item = copy.deepcopy(ori_item)
    keys = [x for x in new_item['match'].keys() if x != '图文']
    new_item['title'] = re.sub(r'\d{1,4}年','', new_item['title'])
    if len(keys) != 0:
        change_num = 0
        for key in keys:
            all_key_values = attr_key[key]
            ori_value = new_item['key_attr'][key]
            key_index = 0
            for i in range(len(all_key_values)):
                if ori_value in all_key_values[i]:
                    key_index = i
                    break
            sim_key_values = all_key_values[key_index]
            if sim_key_values[0] != ori_value:
                new_item['title'] = new_item['title'].replace(ori_value, sim_key_values[0], 1)
                new_item['key_attr'][key] = sim_key_values[0]
                change_num+=1
    return new_item

count = 0
math_list = []
not_math_list = []
with open('../data/train_fine.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    new_dic = {}
    for line in lines:
        data = json.loads(line)
        # math_list.append(json.dumps(data, ensure_ascii=False)+'\n')
        new_item = get_uni_pos_item(data, attr_key)
        new_item is not None and math_list.append(json.dumps(new_item, ensure_ascii=False)+'\n')
        new_item = get_neg_item(new_item, attr_key)
        new_item is not None and not_math_list.append(json.dumps(new_item, ensure_ascii=False)+'\n')
        # new_item = get_neg_item(data, attr_key)
        # new_item is not None and not_math_list.append(json.dumps(new_item, ensure_ascii=False)+'\n')
        # count+=1
        # if count >20:
        #     break

with open('../data/preprocessed_data/uni_pos_fine.txt', 'w', encoding='utf-8') as f:
    f.writelines(math_list)
with open('../data/preprocessed_data/uni_neg_fine.txt', 'w', encoding='utf-8') as f:
    f.writelines(not_math_list)

KeyboardInterrupt: 

In [5]:
with open('../data/preprocessed_data/attr_match.json', 'r', encoding='utf-8') as f:
    attr_key = json.load(f)
attr_key

{'领型': [['高领', '半高领', '立领'],
  ['连帽', '可脱卸帽'],
  ['翻领', '衬衫领', 'POLO领', '方领', '娃娃领', '荷叶领'],
  ['双层领'],
  ['西装领'],
  ['U型领'],
  ['一字领'],
  ['围巾领'],
  ['堆堆领'],
  ['V领'],
  ['棒球领'],
  ['圆领'],
  ['斜领'],
  ['亨利领']],
 '袖长': [['短袖', '五分袖'], ['九分袖', '长袖'], ['七分袖'], ['无袖']],
 '衣长': [['超短款', '短款', '常规款'], ['长款', '超长款'], ['中长款']],
 '版型': [['修身型', '标准型'], ['宽松型']],
 '裙长': [['短裙', '超短裙'], ['中裙', '中长裙'], ['长裙']],
 '穿着方式': [['套头'], ['开衫']],
 '类别': [['手提包'], ['单肩包'], ['斜挎包'], ['双肩包']],
 '裤型': [['O型裤', '锥形裤', '哈伦裤', '灯笼裤'],
  ['铅笔裤', '直筒裤', '小脚裤'],
  ['工装裤'],
  ['紧身裤'],
  ['背带裤'],
  ['喇叭裤', '微喇裤'],
  ['阔腿裤']],
 '裤长': [['短裤'], ['五分裤'], ['七分裤'], ['九分裤', '长裤']],
 '裤门襟': [['松紧'], ['拉链'], ['系带']],
 '闭合方式': [['松紧带'], ['拉链'], ['套筒', '套脚', '一脚蹬'], ['系带'], ['魔术贴'], ['搭扣']],
 '鞋帮高度': [['高帮', '中帮'], ['低帮']]}

In [7]:
# 统一字典
uni_dict = {}
for key, all_attr in attr_key.items():
    attrs = []
    for item_attr in all_attr:
        attrs.append(item_attr[0])
    uni_dict[key]=attrs
with open('../data/preprocessed_data/uni_attr_match.json', 'w', encoding='utf-8') as f:
    f.writelines([json.dumps(uni_dict, ensure_ascii=False)+'\n'])
uni_dict

{'领型': ['高领',
  '连帽',
  '翻领',
  '双层领',
  '西装领',
  'U型领',
  '一字领',
  '围巾领',
  '堆堆领',
  'V领',
  '棒球领',
  '圆领',
  '斜领',
  '亨利领'],
 '袖长': ['短袖', '九分袖', '七分袖', '无袖'],
 '衣长': ['超短款', '长款', '中长款'],
 '版型': ['修身型', '宽松型'],
 '裙长': ['短裙', '中裙', '长裙'],
 '穿着方式': ['套头', '开衫'],
 '类别': ['手提包', '单肩包', '斜挎包', '双肩包'],
 '裤型': ['O型裤', '铅笔裤', '工装裤', '紧身裤', '背带裤', '喇叭裤', '阔腿裤'],
 '裤长': ['短裤', '五分裤', '七分裤', '九分裤'],
 '裤门襟': ['松紧', '拉链', '系带'],
 '闭合方式': ['松紧带', '拉链', '套筒', '系带', '魔术贴', '搭扣'],
 '鞋帮高度': ['高帮', '低帮']}

In [8]:
# 统一字典
dict_to_unit = {}
for key, all_attr in attr_key.items():
    for item_attr in all_attr:
        for val in item_attr:
            dict_to_unit[f'{key}-{val}'] = item_attr[0]
 
with open('../data/preprocessed_data/dict_to_unit.json', 'w', encoding='utf-8') as f:
    f.writelines([json.dumps(dict_to_unit, ensure_ascii=False)+'\n'])\

dict_to_unit

{'领型-高领': '高领',
 '领型-半高领': '高领',
 '领型-立领': '高领',
 '领型-连帽': '连帽',
 '领型-可脱卸帽': '连帽',
 '领型-翻领': '翻领',
 '领型-衬衫领': '翻领',
 '领型-POLO领': '翻领',
 '领型-方领': '翻领',
 '领型-娃娃领': '翻领',
 '领型-荷叶领': '翻领',
 '领型-双层领': '双层领',
 '领型-西装领': '西装领',
 '领型-U型领': 'U型领',
 '领型-一字领': '一字领',
 '领型-围巾领': '围巾领',
 '领型-堆堆领': '堆堆领',
 '领型-V领': 'V领',
 '领型-棒球领': '棒球领',
 '领型-圆领': '圆领',
 '领型-斜领': '斜领',
 '领型-亨利领': '亨利领',
 '袖长-短袖': '短袖',
 '袖长-五分袖': '短袖',
 '袖长-九分袖': '九分袖',
 '袖长-长袖': '九分袖',
 '袖长-七分袖': '七分袖',
 '袖长-无袖': '无袖',
 '衣长-超短款': '超短款',
 '衣长-短款': '超短款',
 '衣长-常规款': '超短款',
 '衣长-长款': '长款',
 '衣长-超长款': '长款',
 '衣长-中长款': '中长款',
 '版型-修身型': '修身型',
 '版型-标准型': '修身型',
 '版型-宽松型': '宽松型',
 '裙长-短裙': '短裙',
 '裙长-超短裙': '短裙',
 '裙长-中裙': '中裙',
 '裙长-中长裙': '中裙',
 '裙长-长裙': '长裙',
 '穿着方式-套头': '套头',
 '穿着方式-开衫': '开衫',
 '类别-手提包': '手提包',
 '类别-单肩包': '单肩包',
 '类别-斜挎包': '斜挎包',
 '类别-双肩包': '双肩包',
 '裤型-O型裤': 'O型裤',
 '裤型-锥形裤': 'O型裤',
 '裤型-哈伦裤': 'O型裤',
 '裤型-灯笼裤': 'O型裤',
 '裤型-铅笔裤': '铅笔裤',
 '裤型-直筒裤': '铅笔裤',
 '裤型-小脚裤': '铅笔裤',
 '裤型-工装裤': '工装裤',
 '裤型-紧身裤': '紧身裤',
 '裤型-背带裤': 

In [9]:
# 负例采样字典
unit_neg_dict = {}
for key, all_attr in uni_dict.items():
    for item_attr in all_attr:
        new_all_attr = all_attr.copy()
        new_all_attr.remove(item_attr)
        unit_neg_dict[f'{key}-{item_attr}'] = new_all_attr
with open('../data/preprocessed_data/unit_neg_sample_dict.json', 'w', encoding='utf-8') as f:
    f.writelines([json.dumps(unit_neg_dict, ensure_ascii=False)+'\n'])

unit_neg_dict

{'领型-高领': ['连帽',
  '翻领',
  '双层领',
  '西装领',
  'U型领',
  '一字领',
  '围巾领',
  '堆堆领',
  'V领',
  '棒球领',
  '圆领',
  '斜领',
  '亨利领'],
 '领型-连帽': ['高领',
  '翻领',
  '双层领',
  '西装领',
  'U型领',
  '一字领',
  '围巾领',
  '堆堆领',
  'V领',
  '棒球领',
  '圆领',
  '斜领',
  '亨利领'],
 '领型-翻领': ['高领',
  '连帽',
  '双层领',
  '西装领',
  'U型领',
  '一字领',
  '围巾领',
  '堆堆领',
  'V领',
  '棒球领',
  '圆领',
  '斜领',
  '亨利领'],
 '领型-双层领': ['高领',
  '连帽',
  '翻领',
  '西装领',
  'U型领',
  '一字领',
  '围巾领',
  '堆堆领',
  'V领',
  '棒球领',
  '圆领',
  '斜领',
  '亨利领'],
 '领型-西装领': ['高领',
  '连帽',
  '翻领',
  '双层领',
  'U型领',
  '一字领',
  '围巾领',
  '堆堆领',
  'V领',
  '棒球领',
  '圆领',
  '斜领',
  '亨利领'],
 '领型-U型领': ['高领',
  '连帽',
  '翻领',
  '双层领',
  '西装领',
  '一字领',
  '围巾领',
  '堆堆领',
  'V领',
  '棒球领',
  '圆领',
  '斜领',
  '亨利领'],
 '领型-一字领': ['高领',
  '连帽',
  '翻领',
  '双层领',
  '西装领',
  'U型领',
  '围巾领',
  '堆堆领',
  'V领',
  '棒球领',
  '圆领',
  '斜领',
  '亨利领'],
 '领型-围巾领': ['高领',
  '连帽',
  '翻领',
  '双层领',
  '西装领',
  'U型领',
  '一字领',
  '堆堆领',
  'V领',
  '棒球领',
  '圆领',
  '斜领',
  '亨利领'],
 '领型-堆堆领': ['高领',
  '连帽'

In [10]:
from tqdm import tqdm
# remove years [fine]
fine_data = '../data/train_fine.txt'
new_fine_data = '../data/preprocessed_data/unit_train_fine.txt'

rets = []
i = 0
with open(fine_data, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        data['title'] = re.sub(r'\d{1,4}年','', data['title'])

        # unit attr
        keys = [x for x in data['match'].keys() if x != '图文']
        for key in keys:
            ori_value = data['key_attr'][key]
            new_value = dict_to_unit[f'{key}-{ori_value}']
            data['title'] = data['title'].replace(ori_value, new_value, 1)
            data['key_attr'][key] = new_value
        rets.append(json.dumps(data, ensure_ascii=False)+'\n')
        
with open(new_fine_data, 'w') as f:
    f.writelines(rets)
print(len(rets))

50000it [01:55, 431.81it/s]


50000


In [28]:
with open(new_fine_data, 'w') as f:
    f.writelines(rets)

In [11]:
import re
import json
import torch
import numpy as np
from tqdm import tqdm 
import itertools
def load_attr_dict(file):
    # 读取属性字典
    with open(file, 'r') as f:
        attr_dict = {}
        for attr, attrval_list in json.load(f).items():
            attrval_list = list(map(lambda x: x.split('='), attrval_list))
            attr_dict[attr] = list(itertools.chain.from_iterable(attrval_list))
    return attr_dict

def match_attrval(title, attr, attr_dict):
    # 在title中匹配属性值
    attrvals = "|".join(attr_dict[attr])
    ret = re.findall(attrvals, title)
    # return "{}{}".format(attr, ''.join(ret))
    return "{}".format(''.join(ret))  

# load attribute dict
attr_dict_file = "../data/attr_to_attrvals.json"
attr_dict = load_attr_dict(attr_dict_file)

In [12]:
# remove years and get attributes [coarse]
# 裤门襟和鞋带的属性有重合，所以需要额外的判断机制
coarse_data = '../data/train_coarse.txt'
new_coarse_data = '../data/preprocessed_data/unit_coarse_fine.txt'
new_neg_coarse_data = '../data/preprocessed_data/unit_neg_coarse_fine.txt'

coarse_rets = []
neg_coarse_rets = []
querys = attr_dict.keys()
i = 0
with open(coarse_data, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        data['title'] = re.sub(r'\d{1,4}年','', data['title'])
        
        if data['match']['图文'] == 1:
            for query in querys:
                values = attr_dict[query]
                if (query == '裤门襟') and ('裤' not in data['title'] ):
                    continue
                if (query == '闭合方式') and ('裤' in data['title'] ):
                    continue
                for value in values:
                    if value in data['title']:
                        data['key_attr'][query] = value
                        data['match'][query] = 1
            # unit attr
            keys = [x for x in data['match'].keys() if x != '图文']
            for key in keys:
                ori_value = data['key_attr'][key]
                new_value = dict_to_unit[f'{key}-{ori_value}']
                data['title'] = data['title'].replace(ori_value, new_value, 1)
                data['key_attr'][key] = new_value
            coarse_rets.append(json.dumps(data, ensure_ascii=False)+'\n') 
        else:
            for query in querys:
                values = attr_dict[query]
                if (query == '裤门襟') and ('裤' not in data['title'] ):
                    continue
                if (query == '闭合方式') and ('裤' in data['title'] ):
                    continue
                for value in values:
                    if value in data['title']:
                        data['key_attr'][query] = value
                        data['match'][query] = -1
            keys = [x for x in data['match'].keys() if x != '图文']
            for key in keys:
                ori_value = data['key_attr'][key]
                new_value = dict_to_unit[f'{key}-{ori_value}']
                data['title'] = data['title'].replace(ori_value, new_value, 1)
                data['key_attr'][key] = new_value
            neg_coarse_rets.append(json.dumps(data, ensure_ascii=False)+'\n') 

with open(new_coarse_data, 'w') as f:
    f.writelines(coarse_rets)

with open(new_neg_coarse_data, 'w') as f:
    f.writelines(neg_coarse_rets)
    
print(len(coarse_rets), len(neg_coarse_rets))

100000it [02:49, 591.69it/s]


89588 10412


In [ ]:
title_len = []

data1 = '../data/preprocessed_data/unit_train_fine.txt'
data2 = '../data/preprocessed_data/unit_coarse_fine89588.txt'
data3 = '../data/preprocessed_data/unit_neg_coarse_fine10412.txt'


with open(coarse_data, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)